In [ ]:
pip install torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 13.5 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchmetrics.classification import Accuracy

# CBAM Implementation
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.fc1 = nn.Conv2d(in_channels, in_channels // reduction, kernel_size=1, bias=False)
        self.fc2 = nn.Conv2d(in_channels // reduction, in_channels, kernel_size=1, bias=False)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(torch.mean(x, dim=(2, 3), keepdim=True))))
        max_out = self.fc2(self.relu(self.fc1(torch.amax(x, dim=(2, 3), keepdim=True))))
        out = avg_out + max_out
        return self.sigmoid(out) * x

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(out)
        return self.sigmoid(out) * x

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, reduction)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        return x

# AlexNet without CBAM
class AlexNet(nn.Module):
    def __init__(self, num_classes=100):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# AlexNet with CBAM
class AlexNet_CBAM(nn.Module):
    def __init__(self, num_classes=100):
        super(AlexNet_CBAM, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            CBAM(64),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            CBAM(192),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            CBAM(384),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            CBAM(256),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# CIFAR-100 Dataset Preparation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for AlexNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Initialize models, loss function, and optimizers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

alexnet = AlexNet(num_classes=100).to(device)
alexnet_cbam = AlexNet_CBAM(num_classes=100).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_alexnet = optim.Adam(alexnet.parameters(), lr=0.001)
optimizer_alexnet_cbam = optim.Adam(alexnet_cbam.parameters(), lr=0.001)

# Top-1 Accuracy
top1_acc = Accuracy(task="multiclass", num_classes=100, top_k=1).to(device)

# Evaluation Function
def evaluate(model, testloader):
    model.eval()
    top1 = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            top1 += top1_acc(outputs, labels)
            total += labels.size(0)
    return top1 / total

# Training Function with Current and Best Accuracy for Each Epoch
def train_model(model, trainloader, testloader, optimizer, num_epochs=20):
    best_top1_acc = 0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        epoch_top1_acc = 0.0
        total = 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate accuracy for each batch
            batch_top1 = top1_acc(outputs, labels)

            epoch_top1_acc += batch_top1
            total += labels.size(0)

        # Average accuracy for the epoch
        epoch_top1_acc /= total

        # Evaluate the model on the test set
        current_top1_acc = evaluate(model, testloader)

        # Update best accuracy
        if current_top1_acc > best_top1_acc:
            best_top1_acc = current_top1_acc
            torch.save(model.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Loss: {running_loss/len(trainloader):.4f}")
        print(f"Current Top-1 Accuracy: {current_top1_acc:.4f}")
        print(f"Best Top-1 Accuracy: {best_top1_acc:.4f}")
        print(f"Epoch Top-1 Accuracy: {epoch_top1_acc:.4f}")

# Choose model and optimizer for training
print("Training AlexNet without CBAM")
train_model(alexnet, trainloader, testloader, optimizer_alexnet, num_epochs=20)

print("Training AlexNet with CBAM")
train_model(alexnet_cbam, trainloader, testloader, optimizer_alexnet_cbam, num_epochs=20)


Files already downloaded and verified
Files already downloaded and verified
Training AlexNet without CBAM
Epoch [1/20]
Loss: 4.1851
Current Top-1 Accuracy: 0.0012
Best Top-1 Accuracy: 0.0012
Epoch Top-1 Accuracy: 0.0008
Epoch [2/20]
Loss: 3.6408
Current Top-1 Accuracy: 0.0018
Best Top-1 Accuracy: 0.0018
Epoch Top-1 Accuracy: 0.0021
Epoch [3/20]
Loss: 3.3788
Current Top-1 Accuracy: 0.0022
Best Top-1 Accuracy: 0.0022
Epoch Top-1 Accuracy: 0.0029
